# Step 1 - Install packages

In [ ]:
import keras
import pandas as pd
import numpy as np
import cv2 as cv
import tensorflow as tf

import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Step 2 - Data Preprocessing & Gether 


## Gather data

In [ ]:
! mkdir ~/.kaggle
! cp /content/kaggle.json ~/.kaggle
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# ! kaggle competitions download -c dog-breed-identification

In [ ]:
# ! unzip /content/dog-breed-identification.zip

In [ ]:
# ! rm /content/dog-breed-identification.zip
# ! rm sample_submission.csv

## Data Preprocessing

In [ ]:
df = pd.read_csv('./labels.csv')

In [ ]:
breed_names = df['breed'].value_counts()[:5].index
breed_names = breed_names.to_list()

In [ ]:
OUTPUT = len(breed_names)

In [ ]:
OUTPUT

In [ ]:
df = df[(df['breed'].isin(breed_names))]

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
label = df['breed'].values

In [ ]:
breed_labels = LabelEncoder()
breed_labels.fit(label)
y = breed_labels.transform(label)
y = pd.get_dummies(y)

In [ ]:
y.shape

In [ ]:
# creating x (independent data)
x = np.zeros((df.shape[0], 224, 224, 3), dtype='float32')

In [ ]:
for i in range(len(df)):
  img = plt.imread(f"./train/{df['id'][i]}.jpg")
  img = cv.resize(img, (224, 224))
  resize = np.expand_dims(img.copy(), axis=0)

  x[i] = resize

In [ ]:
# simplifizing data
X = x/255.0

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
# Remove Variable
del df
del y
del X
del x
del img
del resize
del breed_names

In [ ]:
X_train.shape

## Building Model and Evaluation

In [ ]:
model = keras.models.Sequential()

In [ ]:
# Adding Cnn Layers
model.add(keras.layers.Conv2D(filters=128, kernel_size=3, input_shape=(224, 224, 3)))
model.add(keras.layers.MaxPool2D(pool_size=(2, 2)))
model.add(keras.layers.Dropout(0.2))
# Adding Cnn Layers
model.add(keras.layers.Conv2D(filters=64, kernel_size=4))
model.add(keras.layers.MaxPool2D(pool_size=(2, 2)))
model.add(keras.layers.Dropout(0.3))
# Adding Cnn Layers
model.add(keras.layers.Conv2D(filters=32, kernel_size=3))
model.add(keras.layers.MaxPool2D(pool_size=(2, 2)))
model.add(keras.layers.Dropout(0.4))
# Adding Cnn Layers
model.add(keras.layers.Conv2D(filters=8, kernel_size=2))
model.add(keras.layers.MaxPool2D(pool_size=(2, 2)))
model.add(keras.layers.Dropout(0.5))

In [ ]:
# flatting layers
model.add(keras.layers.Flatten())

In [ ]:
# outopt layes
model.add(keras.layers.Dense(units=128, activation='relu'))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Dense(units=64, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(units=OUTPUT, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
# compile the model
opt = tf.keras.optimizers.Adam(0.0001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# training the model

history = model.fit(X_train, y_train, batch_size=10, epochs=100, validation_data=(X_test, y_test))

In [ ]:
loss, accuracy = model.evaluate(X_train, y_train)

In [ ]:
y_loss, y_accuracy = model.evaluate(X_test, y_test)